In [4]:
!pip install -qU gigachain==0.2.6 gigachain_community==0.2.6 gigachain-cli==0.0.25 langgraph==0.2.19 langchain-community==0.2.16 langchain-openai==0.1.23  python-pptx==0.6.23 python-pptx-interface==0.0.12 python-dotenv==1.0.1 duckduckgo-search==6.2.4 faiss-cpu

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
poetry 1.1.15 requires packaging<21.0,>=20.4, but you have packaging 24.1 which is incompatible.
poetry 1.1.15 requires requests-toolbelt<0.10.0,>=0.9.1, but you have requests-toolbelt 1.0.0 which is incompatible.
tensorflow-intel 2.13.0 requires numpy<=1.24.3,>=1.22, but you have numpy 1.26.4 which is incompatible.
tensorflow-intel 2.13.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.
tensorflow-intel 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.12.0 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import getpass
import os
import requests
import json
import time

from langchain.chat_models.gigachat import GigaChat
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory, ConversationSummaryMemory, VectorStoreRetrieverMemory
from langchain.chains import ConversationChain
from langchain_community.embeddings import GigaChatEmbeddings

from langchain.schema import HumanMessage, SystemMessage
from langchain.tools import tool

# import faiss
from langchain_community.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS

from pydantic import BaseModel

In [2]:
from dotenv import load_dotenv
# Load environment variables
load_dotenv('.env')

sb_auth_data = os.getenv('SB_AUTH_DATA')

# 1. Memory

Define your own class implementing a simple LLM-based chatbot. You need to use at least three memory types (langchain.memory), which are set as one argument in the ```init``` definition. If the memory type has any parameters, you also need to define them as arguments in the ```init``` definition. You also need to define a ```run``` method implementing the main conversation loop, and a ```print_memory``` method to print out what exactly the memory consists of.

Firstly so sorry for badly organised code, its a shame(

In [22]:
class SimpleChatBot:
    def __init__(self, llm, memory_type, memory_window_size):
        # llm, memory_type,
        self.memory_type = memory_type
        self.llm = llm
        # n = {**kwargs.get('memory_window_size')}
        # dict_memory = {'summary': ConversationSummaryMemory(llm=self.llm, memory_window_size=memory_window_size), 'window': ConversationBufferWindowMemory(memory_window_size=memory_window_size), 'simple': ConversationBufferMemory(memory_window_size=memory_window_size)}
        if memory_type == 'summary':
            self.memory = ConversationSummaryMemory(llm=self.llm, memory_window_size=memory_window_size)
        elif memory_type == 'window':
            self.memory = ConversationBufferWindowMemory(memory_window_size=memory_window_size)
        elif memory_type == 'simple':
            self.memory = ConversationBufferMemory(memory_window_size=memory_window_size)

    def _respond(self, user_input, ai_output):
        # TODO
        # self.user_input = user_input
        self.memory.save_context({"input": user_input}, {"output": ai_output})
    

    def print_memory(self):

        return self.memory.load_memory_variables({})


    def run(self):
        print("This is a simple chat bot:")
        
        conversation = ConversationChain(
            llm=self.llm,
            verbose=True,
            memory=self.memory
        )
        while True:
            user_input = input("User: ")
            if user_input == "":
                break
            print(f"User: {user_input}")
            ai_output = conversation.predict(input=user_input)
            print(ai_output)
            self._respond(user_input, ai_output)
            

Now let's check how it works with each type of memory

In [24]:
giga_key = os.environ.get("SB_AUTH_DATA")
giga = GigaChat(credentials=giga_key, model="GigaChat", timeout=30, verify_ssl_certs=False)
chat = SimpleChatBot(giga, 'summary', memory_window_size=2)
chat.run()
chat.print_memory()

This is a simple chat bot:
User: Расскажи о Казани


> Entering new ConversationChain chain...
Prompt after formatting:
Ниже приводится дружеский разговор между человеком и AI. AI разговорчив и предоставляет множество конкретных деталей из своего контекста. Если AI не знает ответа на вопрос, он честно говорит, что не знает.

Текущий разговор:

Human: Расскажи о Казани
AI:

> Finished chain.
Казань — это город в России, столица Республики Татарстан. Он расположен на реке Волге и является важным культурным, экономическим и образовательным центром страны. В Казани можно найти множество исторических и архитектурных достопримечательностей, таких как Казанский Кремль, мечеть Кул-Шариф и башня Сююмбике. Город также известен своей богатой историей и традициями, включая кухню и народные ремесла.
User: Что можно поесть в Казани?


> Entering new ConversationChain chain...
Prompt after formatting:
Ниже приводится дружеский разговор между человеком и AI. AI разговорчив и предоставляет множество ко

{'history': 'AI рассказывает о городе Казани, описывая его как важный культурный, экономический и образовательный центр России с множеством исторических и архитектурных достопримечательностей, таких как Казанский Кремль, мечеть Кул-Шариф и башня Сююмбике. Обсуждается татарская кухня, включая такие блюда, как чак-чак, эчпочмак, кыстыбый, коймак, шулпа, казы и татарский чай. Human интересуется, какие еще интересные города в Татарстане, и AI перечисляет Болгар, Елабугу и Набережные Челны, описывая их историческое значение и культурное наследие.'}

In [23]:
giga_key = os.environ.get("SB_AUTH_DATA")
giga = GigaChat(credentials=giga_key, model="GigaChat", timeout=30, verify_ssl_certs=False)
chat = SimpleChatBot(giga, 'window', memory_window_size=2)
chat.run()
chat.print_memory()

This is a simple chat bot:
User: Расскажи о Екатеринбурге


> Entering new ConversationChain chain...
Prompt after formatting:
Ниже приводится дружеский разговор между человеком и AI. AI разговорчив и предоставляет множество конкретных деталей из своего контекста. Если AI не знает ответа на вопрос, он честно говорит, что не знает.

Текущий разговор:

Human: Расскажи о Екатеринбурге
AI:

> Finished chain.
Екатеринбург — это город в России, расположенный на восточном склоне Уральских гор. Он является административным центром Свердловской области и одним из крупнейших экономических, культурных и научных центров Урала. Население города составляет около 1,5 миллиона человек.

Екатеринбург имеет богатую историю, которая началась в 1723 году с основания металлургического завода. Впоследствии завод превратился в крупный промышленный центр, а сам город стал важным транспортным узлом и торговым пунктом.

В Екатеринбурге находится множество достопримечательностей, включая Храм-на-Крови, построенн

{'history': 'Human: Расскажи о Екатеринбурге\nAI: Екатеринбург — это город в России, расположенный на восточном склоне Уральских гор. Он является административным центром Свердловской области и одним из крупнейших экономических, культурных и научных центров Урала. Население города составляет около 1,5 миллиона человек.\n\nЕкатеринбург имеет богатую историю, которая началась в 1723 году с основания металлургического завода. Впоследствии завод превратился в крупный промышленный центр, а сам город стал важным транспортным узлом и торговым пунктом.\n\nВ Екатеринбурге находится множество достопримечательностей, включая Храм-на-Крови, построенный на месте, где был расстрелян последний российский император Николай II и его семья. Также стоит посетить Плотинку — исторический центр города, где можно прогуляться вдоль реки Исеть и насладиться архитектурой XIX века.\n\nЕсли у вас есть еще вопросы о Екатеринбурге или о чем-то другом, я с радостью отвечу!\nHuman: Какие там есть университеты?\nAI: В

In [25]:
giga_key = os.environ.get("SB_AUTH_DATA")
giga = GigaChat(credentials=giga_key, model="GigaChat", timeout=30, verify_ssl_certs=False)
chat = SimpleChatBot(giga, 'simple', memory_window_size=2)
chat.run()
chat.print_memory()

This is a simple chat bot:
User: Расскажи о Томске


> Entering new ConversationChain chain...
Prompt after formatting:
Ниже приводится дружеский разговор между человеком и AI. AI разговорчив и предоставляет множество конкретных деталей из своего контекста. Если AI не знает ответа на вопрос, он честно говорит, что не знает.

Текущий разговор:

Human: Расскажи о Томске
AI:

> Finished chain.
Томск — это город в Сибири, Россия. Он был основан в 1604 году и является одним из старейших городов в этом регионе. Томск известен своими многочисленными университетами и академическими учреждениями, что делает его центром образования и науки в России. В городе также есть множество исторических зданий и музеев, которые отражают его богатую историю.
User: Какие есть университеты в Томске?


> Entering new ConversationChain chain...
Prompt after formatting:
Ниже приводится дружеский разговор между человеком и AI. AI разговорчив и предоставляет множество конкретных деталей из своего контекста. Если AI

{'history': 'Human: Расскажи о Томске\nAI: Томск — это город в Сибири, Россия. Он был основан в 1604 году и является одним из старейших городов в этом регионе. Томск известен своими многочисленными университетами и академическими учреждениями, что делает его центром образования и науки в России. В городе также есть множество исторических зданий и музеев, которые отражают его богатую историю.\nHuman: Расскажи о Томске\nAI: Томск — это город в Сибири, Россия. Он был основан в 1604 году и является одним из старейших городов в этом регионе. Томск известен своими многочисленными университетами и академическими учреждениями, что делает его центром образования и науки в России. В городе также есть множество исторических зданий и музеев, которые отражают его богатую историю.\nHuman: Какие есть университеты в Томске?\nAI: В Томске находится несколько известных университетов, включая Томский государственный университет, который является одним из старейших вузов в России, основанный в 1878 году. 

Please make a short report abount differences between used memory types

Report: Summary memory type is quite useful cause you dont have to read whole conversation by yourself, and its not requires lots of memory, Buffer memory consists of all conversation and if you need all details to be save its quite good, Window memory saves last _k_ dialogs and its useful when you need most recent dialogs

# 2. Using tools and agents

## 2.1 Using tools and API

Create your own tool based on the langchain.tools library to interact with a public OpenWeather API. This tool will receive data from the API and return it as a readable result for the user.


OpenWeather API URL: https://api.openweathermap.org/data/2.5/weather?q={city}&appid={openweather_key}&units=metric

[How to get OpenWeather API key](https://docs.google.com/document/d/1vbi8QKqMZqZoCReIzpmEB_2mHsrbmXPlyGngE3jeDDw/edit)


In [12]:
from langchain.tools import Tool
from langchain.agents import create_gigachat_functions_agent, AgentExecutor
import requests
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Optional


openweather_key = os.environ.get("OPENWEATHER_API_KEY")

class WeatherForecastResult(BaseModel):
    condition: str = Field(description="Weather conditions in the given city in particular time")
    temperature: float = Field(description="Degrees of temperature in Celsium")
    message: str = Field(description="Message about the result of the distance calculation")

few_shot_example = [{"request": "What is the weather in Ekaterinburg today?",
                    "params": {'city': "Ekaterinburg"}}]

@tool(few_shot_examples=few_shot_example)
def get_wheather(city: str) -> str:
    """
    Returns weather in given city
    """
    lang = "ru"
    base_url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={openweather_key}&lang={lang}&units=metric"
    try:
        response = requests.get(base_url)
        response.raise_for_status()
        data = response.json()
        degrees = data['main']['temp']
        # temperature = f"{degrees}°С."
        condition = data['weather'][0]['description']
        return WeatherForecastResult(condition=str(condition), temperature=float(degrees), message="Sucessfully got temperature")
    except requests.exceptions.RequestException as e:
        return WeatherForecastResult(condition='', temperature=0.0, message=f"Error getting tempersture: {str(e)}")
    except (KeyError, IndexError):
        return WeatherForecastResult(condition='', temperature=0.0, message="Error in server response data.")

class OpenWeatherAPITool:
    def __init__(self, llm, agent_function):
        self.llm = llm
        self.agent_function = agent_function

    def run(self, user_input: str):
        tool_wf = [get_wheather]
        agent = create_gigachat_functions_agent(self.llm, tool_wf)
        agent_executor = AgentExecutor(
        agent=agent,
        tools=tool_wf,
        verbose=True,
        )
        try:
            result = agent_executor.invoke(
                {
                    "input": user_input,
                }
            )
            return result['output']
        except Exception as e:
            print(f"Error: {str(e)}")
        time.sleep(0.2)


Let's check it

In [13]:

giga_key = os.environ.get("SB_AUTH_DATA")
giga_pro = GigaChat(credentials=giga_key, model="GigaChat-Pro", timeout=30, verify_ssl_certs=False)

openwheatertool = OpenWeatherAPITool(giga_pro, get_wheather)
user_input = "Какая погода сейчас в Ставрополе?"
openwheatertool.run(user_input)



> Entering new AgentExecutor chain...

Invoking: `get_wheather` with `{'city': 'Ставрополь'}`


condition='пасмурно' temperature=4.16 message='Sucessfully got temperature'Сейчас в Ставрополе пасмурно, температура около 4 градусов Цельсия.

> Finished chain.


'Сейчас в Ставрополе пасмурно, температура около 4 градусов Цельсия.'

{'input': 'Какая погода сейчас в Ставрополе?',
 'output': 'В Ставрополе сейчас ясно, температура 18.16°С.'}

## 2.2. Multi agents

Create a multi-agent system where each agent is responsible for a specific task in the travel planning process. For example, one agent is responsible for searching for flights, another for booking hotels, and a third for finding the weather at the destination.

Requirements:

- Use three or more GigaChat-based agents to interact with each other.
- The first agent is responsible for searching for flights (using ```get_url_booking_tickets``` function).
- The second agent is responsible for booking hotels (using ```get_url_booking_hotels``` function).
- The third agent collects weather information for the destination (using a real API, such as OpenWeather). You can use the function from the previous task (for simplify, here you can give a current weather, not a forecast for the specific date)

In [38]:
from langchain.tools import Tool
from langchain.agents import create_gigachat_functions_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import requests
import json
import datetime
from dateutil import parser

openweather_key = os.environ.get("OPENWEATHER_API_KEY")

def get_geoid(city: str) -> str:
    url_base = 'https://suggest-maps.yandex.ru/suggest-geo'
    params = {'search_type': 'tune', 'v': '9', 'results': 1, 'lang': 'ry_RU', 'callback': 'json'}
    params['part'] = city
    r = requests.get(url_base, params=params)
    if r.ok:
        r_text = r.text
        r_json = r_text[5: len(r_text)-1]
        res_json = json.loads(r_json)
        res = res_json['results'][0]['geoid']
    else:
        res = ''
    return str(res)

few_shot_hotels = [{"request": "Организуй поездку в Екатеринбург с 10.12.2024 по 22.12.2024 - отель, самолет, погода", "params": {"date_in_str": "10.12.2024", "date_out_str": "22.12.2024", "city": "Екатеринбург"}}]
#TODO
# First
class HotelsBookingResult(BaseModel):
    link: str = Field(description="Link to hotel booking")
    message: str = Field(description="We found good option for you")

@tool(few_shot_examples=few_shot_hotels)
def get_url_booking_hotels(date_in_str: str, date_out_str: str, city: str) -> str:
    #TODO
    """
    Returns link to a hotels booking on given period of time
    """
    date_in = parser.parse(date_in_str)
    date_out = parser.parse(date_out_str)
    if date_in is None:
        date_in = datetime.datetime.now()
    if date_out is None:
        date_out = datetime.datetime.now() + datetime.timedelta(days=1)
    geoid = get_geoid(city)
    url = 'https://travel.yandex.ru/hotels/search/?'
    params = {'adults': '2', 'checkinDate': date_in.strftime('%Y-%m-%d'), 'checkoutDate': date_out.strftime('%Y-%m-%d'), 'childrenAges': '0', 'geoId': geoid}
    for item in params:
        url += '&' + item + '=' + params[item]
    # return f'Here is your URL for booking: {url} in {city} on {date_in_str} / {date_out_str}'
    return HotelsBookingResult(link=url, message=f"Here is your URL for booking: in {city} on {date_in_str} / {date_out_str}" )



few_shot_tickets = [{"request": "Организуй поездку в Екатеринбург из Томска с 10.12.2024 по 22.12.2024 - отель, самолет, погода", "params": {"date_in_str": "10.12.2024", "date_out_str": "22.12.2024","city_from": "Томск","city_to": "Екатеринбург"}}]
#TODO
# Second
class TicketsBookingResult(BaseModel):
    link: str = Field(description="Link for a flight")
    message: str = Field(description="We found good option for you")

@tool(few_shot_examples=few_shot_tickets)
def get_url_booking_tikets(city_from: str, city_to: str, date_in_str: str, date_out_str: str) -> str:
    """
    Returns link for a flight from city A to B on given dates
    """
    #TODO
    date_in = parser.parse(date_in_str)
    date_out = parser.parse(date_out_str)
    if date_in is None:
        date_in = datetime.datetime.now()
    if date_out is None:
        date_out = datetime.datetime.now() + datetime.timedelta(days=1)
    fromid = get_geoid(city_from)
    toid = get_geoid(city_to)
    url = 'https://travel.yandex.ru/avia/search/result/?'
    params = {'adults_seats': '2', 'fromId': 'c' + fromid, 'klass': 'economy', 'oneway': '2', 'return_date': date_out.strftime('%Y-%m-%d'), 'toId': 'c' + toid, 'when': date_in.strftime('%Y-%m-%d')}
    for item in params:
        url += '&' + item + '=' + params[item]
    # return f'Here is your url for tickets ordering: {url} from {city_from} to {city_to} on {date_in_str} / {date_out_str}'
    return TicketsBookingResult(link=url, message=f"Here is your url for tickets ordering: from {city_from} to {city_to} on {date_in_str} / {date_out_str}")



few_shot_weather = [{"request": "Организуй поездку в Екатеринбург из Томска с 10.12.2024 по 22.12.2024 - отель, самолет, погода","params": {'city': "Екатеринбург"}}]

@tool(few_shot_examples=few_shot_example)
def get_wheather_booking(city: str) -> str:
    """
    Returns the weather in given city
    """
    lang = "ru"
    base_url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={openweather_key}&lang={lang}&units=metric"
    try:
        response = requests.get(base_url)
        response.raise_for_status()
        data = response.json()
        degrees = data['main']['temp']
        condition = data['weather'][0]['description']
        return WeatherForecastResult(condition=str(condition), temperature=float(degrees), message="Sucessfully got temperature")
    except requests.exceptions.RequestException as e:
        return WeatherForecastResult(condition='', temperature=0.0, message=f"Error getting tempersture: {str(e)}")
    except (KeyError, IndexError):
        return WeatherForecastResult(condition='', temperature=0.0, message="Error in server response data.")
    


class MultiAgent:
    def __init__(self, llm, agent_function_wheater, agent_function_hotels, agent_function_tickets):
        self.llm = llm
        self.agent_function_wheater = agent_function_wheater
        self.agent_function_hotels = agent_function_hotels
        self.agent_function_tickets = agent_function_tickets
        
        
    def _create_agent(self, tools:list, system_prompt):
                
        agent = create_gigachat_functions_agent(self.llm, tools)

        # agent_executor = AgentExecutor(
        #     agent=agent,
        #     tools=tools,
        #     verbose=True,
        # )
        return AgentExecutor(
            agent=agent,
            tools=tools,
            verbose=True,
            )

    def run(self, user_input: str):
        #TODO
        tools = [self.agent_function_wheater, self.agent_function_hotels, self.agent_function_tickets]
        memory = ConversationBufferWindowMemory(memory_window_size=2)
        system_prompt = [
        SystemMessage(content="You're a useful assistant. Follow the user's commands, clarify the missing data.")
    ]
        booking_agent = self._create_agent(tools, system_prompt)
        try:
            result = booking_agent.invoke(
                {
                    "input": user_input,
                }
            )
            # we will save user activity he he he
            memory.save_context({"input":user_input}, {"output": result['output']})
            # memory.append(AIMessage(content=result["output"]))
            return result['output']
            # return answer
        except Exception as e:
            print(f"Error: {str(e)}")
        time.sleep(0.2)


In [39]:
giga_key = os.environ.get("SB_AUTH_DATA")
giga_pro = GigaChat(credentials=giga_key, model="GigaChat", timeout=30, verify_ssl_certs=False)

traveler = MultiAgent(giga_pro, get_wheather, get_url_booking_hotels, get_url_booking_tikets)
user_input = "Организуй поездку в Санкт-Петербурге на 10 дней с 15.11.2024 - отель, самолет, погода"
answer = traveler.run(user_input)
print(answer)



> Entering new AgentExecutor chain...

Invoking: `get_url_booking_hotels` with `{'city': 'Санкт-Петербург', 'date_in_str': '15.11.2024', 'date_out_str': '25.11.2024'}`


link='https://travel.yandex.ru/hotels/search/?&adults=2&checkinDate=2024-11-15&checkoutDate=2024-11-25&childrenAges=0&geoId=2' message='Here is your URL for booking: in Санкт-Петербург on 15.11.2024 / 25.11.2024'Вот ваш URL для бронирования проживания в Санкт-Петербурге с 15.11.2024 по 25.11.2024:

https://travel.yandex.ru/hotels/search/?&adults=2&checkinDate=2024-11-15&checkoutDate=2024-11-25&childrenAges=0&geoId=2

> Finished chain.
Вот ваш URL для бронирования проживания в Санкт-Петербурге с 15.11.2024 по 25.11.2024:

https://travel.yandex.ru/hotels/search/?&adults=2&checkinDate=2024-11-15&checkoutDate=2024-11-25&childrenAges=0&geoId=2
